<a href="https://colab.research.google.com/github/QmccfdVbVse5AbfE/onboarding-5pzjmlr1eimp0jjw/blob/master/attempting_to_scrape.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install packages
!pip install -U tmtoolkit
!pip install matplotlib==3.1.1
!pip install svgpath2mpl 

In [ ]:
from collections import defaultdict
import json
import pandas as pd 
import numpy as np 
import requests 
import datetime
from bs4 import BeautifulSoup
import statsmodels.api as sm
from statsmodels.tools import eval_measures
import copy

# Text processing imports
import spacy
parser = spacy.load("en_core_web_sm")
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from tmtoolkit.topicmod.evaluate import metric_coherence_gensim

# Visualization imports
import seaborn as sns
import matplotlib.pyplot as plt
from svgpath2mpl import parse_path

# Statistical test imports
import scipy
import statsmodels
from scipy.stats import friedmanchisquare
from statsmodels.graphics.gofplots import qqplot

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm
/usr/local/lib/python3.7/dist-packages/spacy/util.py:275: UserWarning: [W031] Model 'en_core_web_sm' (2.2.5) requires spaCy v2.2 and is incompatible with the current spaCy version (2.3.7). This may lead to unexpected results or runtime errors. To resolve this, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


# DATA DELIVERABLE (CHECKPOINT 1)

## CONSTRUCT THE DATASET

In [ ]:

# Mount Google Drive locally
from google.colab import drive
drive.mount('/content/drive')

# # Dataset from: https://www.whitehouse.gov/omb/historical-tables/
# # NOTE: make sure the shared projet folder follows this path
# with open('/content/drive/My Drive/CS1951A Final Project/hist05z2_fy22.csv', 'r', encoding='utf-8') as data:
#   budget_df = pd.read_csv(data, header=2, index_col="Department or other unit")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Webscrape State of The Union Addresses
# Dataset from: https://garden.org/plants/group/

speech_table_url = "https://garden.org/plants/group/"
def get_yearly_speeches():
  headers = {'user-agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64; rv:77.0) Gecko/20190101 Firefox/77.0'}
  # proxies = {'http': 'http://194.224.192.226','https': 'http://194.224.192.226',}
  response = requests.get(speech_table_url, headers = headers)
  print(response.text)

  print("headers:", response.request.headers)
  print("response: ", response, type(response))
  # print("response.text: ", response.text, type(response.text))
  # print("response.content: ", response.content, type(response.text))
  bs = BeautifulSoup(response.text, 'html.parser')
  start = False
  all_speeches_url = []
  for link in bs.find_all('a'):
    if link.has_attr('href'):
      if(link.attrs['href'] == '/plants/group/daylilies/'):
        start = True
      if(start):
        all_speeches_url.append('https://garden.org' + link.attrs['href'])
      if(link.attrs['href'] == '/plants/group/zea/'):
        start = False
  return all_speeches_url
all_generic_url = get_yearly_speeches()
print(all_generic_url)

Excessive requests from your IP has resulted in an automatic block being added to our system. This is a protective mechanism to protect us against malicious bot traffic. If you are a regular visitor of the site, then please accept our apologies and know that we did not intend to block you. If you feel that your IP should not have been blocked, please contact us at admin@garden.org.<P>When you contact us, include your IP (which is 35.237.78.181) in your message.
headers: {'user-agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64; rv:77.0) Gecko/20190101 Firefox/77.0', 'Accept-Encoding': 'gzip, deflate', 'Accept': '*/*', 'Connection': 'keep-alive'}
response:  <Response [200]> <class 'requests.models.Response'>
[]


In [ ]:
# print(len(all_generic_url))

def get_generic_id():
  all_generic_links = []
  for curr_link in all_generic_url:
  # for curr_link in all_generic_url:
  # curr_link = all_generic_url[0] #need to change when doing all
    headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/75.0.3770.142 Safari/537.36'}
    response = requests.get(curr_link, headers = headers)
    print("headers:", response.request.headers)
    print("response: ", response, type(response))
    bs = BeautifulSoup(response.text, 'html.parser')
    sections = bs.find_all('div', {"class": "panel-body"})
    if(len(sections) >= 3):
      if(len(sections[2]) > 1):
        specific_link = "https://garden.org" + sections[2].contents[1].attrs['href']
        all_generic_links.append(specific_link)
        print(specific_link)
  
  # print(sections[2].contents[1])
  return all_generic_links
  # link = sections.find('a')
  # if link.has_attr('href'):
  #   print(link.attrs['href'])
test_plant = get_generic_id()


In [ ]:
print(test_plant)

In [ ]:
print(test_plant)
def get_20_popular():
  all_plant_pages = []
  # for curr_link in test_plant:
  headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/75.0.3770.142 Safari/537.36'}
  response = requests.get(test_plant[0] + 'popular/', headers = headers)
  print("headers:", response.request.headers)
  print("response: ", response, type(response))
  bs = BeautifulSoup(response.text, 'html.parser')
  sections = bs.find_all('td')
  print(sections)
  for item in sections:
    print(item.contents)
    all_plant_pages.append(item.attrs['href'])
  return all_plant_pages
# popular = get_20_popular()
# print(popular)


In [ ]:
min_id = 2
max_id = 783799
max_id = 10
all_valid_ids = range(min_id, max_id)
def get_all_subtypes():
  all_plant_pages = []
  base_url = "https://garden.org/plants/view/794278/Chinese-Wisteria-Wisteria-sinensis-Variegata/"
  for i in all_valid_ids:
    whole_url = base_url + '/' + str(i)
    headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/75.0.3770.142 Safari/537.36'}
    response = requests.get(whole_url, headers = headers)
    print("headers:", response.request.headers)
    print("response: ", response, type(response))
    # print("response.text: ", response.text, type(response.text))
    all_plant_pages.append(response.text)

  return all_plant_pages

all_plants = get_all_subtypes()


In [ ]:
# print(all_plants[0])

bs = BeautifulSoup(all_plants[0], 'html.parser')
for link in bs.find_all('tbody'):
  print(link)

In [ ]:
for speech in all_speeches_url:
  # print(speech['president'])
  if(speech['president'] == 'gerald r. ford'):
    break
  response = requests.get(speech['url'])
  html_dump = BeautifulSoup(response.text, 'html.parser')
  # cols = html_tables.find("tbody")
  stocks = html_dump.find_all("section")
  full_text_together = ""
  for i in stocks:
    full_text_broken = i.find_all("p")
    # print("text", full_text_broken)
    for j in full_text_broken:
      # print("text", full_text)
      full_text_together += j.text + " "
      speech['speech'] = (full_text_together)
# print(all_speeches_url)

In [ ]:
# Convert to pandas DF
speeches_df = pd.DataFrame(all_speeches_url)
speeches_df['president'][0] = 'joe biden'
speeches_df.head(48)

Load in health dictionary dataset

In [ ]:
with open('/content/drive/My Drive/CS1951A Final Project/health_dictionary.txt', 'r', encoding='utf-8') as data:
  items =  data.readlines()
  health_dic = [x.strip().lower() for x in items] 
print(health_dic)

health_dic.append(["insurance", "healthcare"])
print(len(health_dic))

# CLEAN AND PROCESS DATA

In [ ]:
# Filter budget_df and transform strings to floats
health_budget_df = budget_df.loc[['Department of Health and Human Services', 'Total budget authority']]
health_budget_df.loc['Department of Health and Human Services'] = health_budget_df.loc['Department of Health and Human Services'].apply(lambda x: float(x.split()[0].replace(',', '')))
health_budget_df.loc['Total budget authority'] = health_budget_df.loc['Total budget authority'].apply(lambda x: float(x.split()[0].replace(',', '')))

# Transform budget_df into percentage (dept of health & human services / total budget authority)
percent_health_budget_series = pd.Series(health_budget_df.loc['Department of Health and Human Services'].divide(health_budget_df.loc['Total budget authority']), name='Percent health budget spend')
transformed_health_budget_df = health_budget_df.append(percent_health_budget_series)

# Drop columns past 2020 (speeches dataset until 2020) and TQ
transformed_health_budget_df = transformed_health_budget_df.drop(labels=['TQ', '2021 estimate', '2022 estimate', '2023 estimate', '2024 estimate', '2025 estimate', '2026 estimate'], axis=1)

transformed_health_budget_df

In [ ]:
print(f"Raw budget df shape: {budget_df.shape}")
print(f"Health budget df shape: {transformed_health_budget_df.shape}")

In [ ]:
print(f"Raw number of speeches: {speeches_df.shape}")
print(f"Raw number of Presidents: {speeches_df.groupby(by=['president'] , axis=0).count()['speech']}")

print(speeches_df[speeches_df['president'] == 'gerald r. ford'])

# Drop N/A rows in speeches data
speeches_df_filtered = speeches_df.dropna(axis=0, subset=['speech'])
# Removes "about search"
speeches_df_filtered["speech"] = speeches_df_filtered["speech"].apply(lambda x: x[14:])

# Filter out Joe Biden from dataframe (see reasoning in write-up)
speeches_df_filtered_with_biden = speeches_df_filtered.copy(deep=True)
speeches_df_filtered = speeches_df_filtered[speeches_df_filtered['president'] != 'joe biden']

print(f"Transformed number of speeches: {speeches_df_filtered.shape}")
print(f"Transformed number of Presidents: {speeches_df_filtered.groupby(by=['president'] , axis=0).count()['speech']}")

In [ ]:
# Include partisanship for each president
conditions = [
              speeches_df_filtered['president'] == 'donald j. trump',
              speeches_df_filtered['president'] == 'barack obama',
              speeches_df_filtered['president'] == 'george w. bush',
              speeches_df_filtered['president'] == 'william j. clinton',
              speeches_df_filtered['president'] == 'george bush',
              speeches_df_filtered['president'] == 'ronald reagan',
              speeches_df_filtered['president'] == 'jimmy carter',
]

is_dem = ['False', 'True', 'False', 'True', 'False', 'False', 'True']
speeches_df_filtered['is_democrat'] = np.select(conditions, is_dem)

speeches_df_filtered.head()